# SP SMIB SynGenTrStab Kundur Example 1

This example is taken from: P. Kundur, "Power System Stability and Control", Example 13.2, pp. 864-869.

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import subprocess

#%matplotlib widget

name = 'SP_SynGenTrStab_SMIB_Fault_KundurExample1'

dpsim_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

path_exec = dpsim_path + '/build/Examples/Cxx/'

timestep = 100e-6

## Run Simulation

In [ ]:
sim = subprocess.Popen([path_exec+name, '--name', name, '--timestep', str(timestep)], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

## Read DPsim Results

In [ ]:
model_name = 'SynGenTrStab_SMIB_Fault_KundurExample1'
path_ref = 'logs/' + 'SP_' + model_name + '_SP/'
dpsim_result_file_ref = path_ref  + 'SP_' + model_name + '_SP.csv'
ts_dpsim_ref = read_timeseries_csv(dpsim_result_file_ref)

## Read PSAT Results

In [ ]:
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/PSAT/Kundur2-Example1/d_kundur2_example1_dpsim.out'
local_file = 'reference-results/d_kundur2_example1_dpsim.out'
urllib.request.urlretrieve(url, local_file) 

timeseries_names_psat = ['delta_Syn_1', 'omega_Syn_1', 'theta_Bus1', 'theta_Bus2', 'theta_Bus3', 'V_Bus1', 'V_Bus2', 'V_Bus3', 'pm_Syn_1', 'vf_Syn_1', 'p_Syn_1', 'q_Syn_1']

ts_psat = read_timeseries_PSAT(local_file, timeseries_names_psat)

## Unit harmonization

In [ ]:
vbase = 400e3 # for p.u. conversion
deg_to_rad = np.pi/180
sbase = 100e6
omega_base = 2*np.pi*60
sgen = 2220e6

## Definition ROI and common timestep

In [ ]:
timestep_common = 100e-6

t_begin=0
t_end=10

begin_idx = int(t_begin/timestep_common)
end_idx= int(t_end/timestep_common)

## Bus voltages

In [ ]:
plt.figure(figsize=(12,8))

for name in ['v1', 'v2', 'v3']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).abs().values[begin_idx:end_idx], label=name + ' SP DPsim')
for name in ['V_Bus1', 'V_Bus2', 'V_Bus3']:
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx]*vbase, label=name + ' SP PSAT', linestyle='--')
plt.legend()
plt.show()

## Bus angles

In [ ]:
plt.figure(figsize=(12,8))

for name in ['v1', 'v2', 'v3']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).phase().values[begin_idx:end_idx]*deg_to_rad, label=name + ' phase SP DPsim')
for name in ['theta_Bus1', 'theta_Bus2', 'theta_Bus3']:
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx], label=name + ' SP PSAT', linestyle='--')
plt.legend()
plt.show()

## Generator emf

In [ ]:
plt.figure(figsize=(12,8))

for name in ['Ep']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).abs().values[begin_idx:end_idx], label=name + ' SP DPsim')
for name in ['vf_Syn_1']:
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx]*vbase, label=name + ' SP PSAT', linestyle='--')
plt.ylim([0.5*vbase,1.5*vbase])
plt.legend()
plt.show()

## Fault Vars (DPsim only)

In [ ]:
plt.figure(figsize=(12,8))
for name in ['v_fault']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).abs().values[begin_idx:end_idx], label=name + 'SP DPsim')
for name in ['i_fault']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).abs().values[begin_idx:end_idx], label=name + 'SP DPsim')

plt.xlim([0.4,0.6])
plt.legend()
plt.show()

## Generator mechanical power

In [ ]:
plt.figure(figsize=(12,8))

for name in ['P_mech']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).values[begin_idx:end_idx], label=name + ' SP DPsim')
for name in ['pm_Syn_1']:
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx]*sbase, label=name + ' SP PSAT', linestyle='--')
plt.legend()
plt.show()

## Generator active power

In [ ]:
plt.figure(figsize=(12,8))

for name in ['P_elec']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).values[begin_idx:end_idx], label=name + ' SP DPsim')
for name in ['p_Syn_1']:
    ts_psat[name].values = ts_psat[name].values*sbase
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx], label=name + ' SP PSAT', linestyle='--')
plt.legend()
plt.show()

## Generator reactive power

In [ ]:
plt.figure(figsize=(12,8))

for name in ['Q_elec']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).values[begin_idx:end_idx], label=name + ' SP DPsim')
for name in ['q_Syn_1']:
    ts_psat[name].values = ts_psat[name].values*sbase
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx], label=name + ' SP PSAT', linestyle='--')
plt.legend()
plt.show()

## Rotor angular velocity $\omega _r$

In [ ]:
plt.figure(figsize=(12,8))

for name in ['wr_gen']:
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).values[begin_idx:end_idx], label='$\omega _r$' + ' SP DPsim')
for name in ['omega_Syn_1']:
    ts_psat[name].values = ts_psat[name].values*omega_base
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx], label='$\omega _r$' + ' SP PSAT', linestyle='--')
plt.legend()
plt.show()

## Rotor angle $\delta _r$

In [ ]:
plt.figure(figsize=(12,8))

for name in ['delta_r_gen']: 
    plt.plot(ts_dpsim_ref[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_dpsim_ref[name].interpolate(timestep_common).values[begin_idx:end_idx], label='$\delta _r$' + ' SP DPsim')
for name in ['delta_Syn_1']:
    plt.plot(ts_psat[name].interpolate(timestep_common).time[begin_idx:end_idx], ts_psat[name].interpolate(timestep_common).values[begin_idx:end_idx], label='$\delta _r$' + ' SP PSAT', linestyle='--')
plt.legend()
plt.show()

## Comparison DPsim vs. PSAT

In [ ]:
p_elec_diff = ts_dpsim_ref['P_elec'].rmse(ts_dpsim_ref['P_elec'].interpolate(timestep_common), ts_psat['p_Syn_1'].interpolate(timestep_common))
print('{:.3f} MVA, which is {:.3f}% of nominal Sgen = {:.2f} MVA'.format(p_elec_diff/1e6, p_elec_diff/sgen*100, sgen/1e6))
q_elec_diff = ts_dpsim_ref['Q_elec'].rmse(ts_dpsim_ref['Q_elec'].interpolate(timestep_common), ts_psat['q_Syn_1'].interpolate(timestep_common))
print('{:.3f} MVA, which is {:.3}% of nominal Sgen = {:.2f} MVA'.format(q_elec_diff/1e6, q_elec_diff/sgen*100,sgen/1e6))
omega_r_diff = ts_dpsim_ref['wr_gen'].rmse(ts_dpsim_ref['wr_gen'].interpolate(timestep_common), ts_psat['omega_Syn_1'].interpolate(timestep_common))
print('{:.3f} 1/s, which is {:.3}% of nominal omega {:.2f} 1/s'.format(omega_r_diff, omega_r_diff/omega_base*100, omega_base))
delta_r_gen_diff = ts_dpsim_ref['delta_r_gen'].rmse(ts_dpsim_ref['delta_r_gen'].interpolate(timestep_common), ts_psat['delta_Syn_1'].interpolate(timestep_common))
print('{:.3} rad'.format(delta_r_gen_diff))

## Assertion DPsim vs. PSAT

In [ ]:
assert(p_elec_diff/1e6<0.6)
assert(q_elec_diff/1e6<4)
assert(omega_r_diff<0.002)
assert(delta_r_gen_diff<2e-4)